# Merging Function for All Datasets

In [3]:
#!pip install pandas
import pandas as pd 

In [4]:
def read_excel_file(file_name, sheet_name):
    
    #loading excel file 
    xls = pd.ExcelFile(file_name)
    
    df = pd.read_excel(xls, sheet_name) # if contains multiple sheets
    
    return df 

In [5]:
def set_df_col(df):
    
    #replacing header with first row 
    
    df.columns = df.iloc[0] 
    df = df[1:]
    
    return df

In [6]:
def add_year(df, year, col_num):
    
    year_col = []

    for i in range(0,len(df)):
        year_col.append(year)
        
    df.insert(col_num, "Year", year_col)
    
    return df

In [7]:
def add_missing_col(df) :
#adding missing columns 
    
    if df['Year'].iloc[0] == '2010' or df['Year'].iloc[0] == '2011' :
        df["% Physically Inactive"] = " "
        
    if df['Year'].iloc[0] == '2011' or df['Year'].iloc[0] == '2012' or df['Year'].iloc[0] == '2013' or df['Year'].iloc[0] == '2014':
        df["Income Ratio"] = " "
        
    return df

In [8]:
def rename_cols(df): 
    
    df.rename(columns={'Unnamed: 0': "FIPS", 'Unnamed: 1': "State", 'Unnamed: 2': "County"}, inplace=True) #renaming columns  
    
    df.rename(columns = {"% Binge Drinking": "% Excessive Drinking"}, inplace = True)
    
    if df.columns.str.contains("% AFGR").any() :
        df.rename(columns = {"% AFGR": "High School Grad Rate"}, inplace = True)
    elif df.columns.str.contains("AFGR").any() :
        df.rename(columns = {"AFGR": "High School Grad Rate"}, inplace = True)
    elif df.columns.str.contains("Graduation Rate").any() :
        df.rename(columns = {"Graduation Rate": "High School Grad Rate"}, inplace = True)
            
    if df.columns.str.contains("PSED").any() :
        df.rename(columns = {"PSED": "% Some College Education"}, inplace = True)
    elif df.columns.str.contains("% PSED").any() :
        df.rename(columns = {"% PSED": "% Some College Education"}, inplace = True)     
    elif df.columns.str.contains("% College").any() :
        df.rename(columns = {"% College": "% Some College Education"}, inplace = True)
    elif df.columns.str.contains("% Some College").any() :
        df.rename(columns = {"% Some College": "% Some College Education"}, inplace = True)

    if df.columns.str.contains("% unemployed").any() :
        df.rename(columns = {"% unemployed": "% Unemployed"}, inplace = True)
        
    if df.columns.str.contains("GINI").any() :
        df.rename(columns = {"GINI": "Income Ratio"}, inplace = True)
    
    return df

In [9]:
def extract_clean_df(file_name, sheet_name, replace_header, year, year_col_num, col_list): #col_list = string, replace_header = bool
    
    #create new empty dataframe for return
    clean_df = pd.DataFrame()
    
    #read file
    df = read_excel_file(file_name, sheet_name) # 0: file name, 1: sheet name
    
    #replacing header
    if replace_header == True:
        df = set_df_col(df)
        
    #renaming columns
    df = rename_cols(df)
    
    #insert year
    df = add_year(df, year, year_col_num)  
    
    #insert missing columns
    df = add_missing_col(df)
     
    #extracting columns 
    clean_df = df[col_list].copy()
    
    return clean_df 

In [10]:
def merge_row(merged_df, df) :

    merged_df = merged_df.append(df)
        
    return merged_df

In [11]:
def del_null_row(df):
    
    #use isnull() pandas function, python None does not work in this case
    
    new_df = df[(df['FIPS'].isnull() == False) & (df['State'].isnull() == False) & (df['County'].isnull() == False)]
    
    return new_df

In [12]:
def del_nullstate_row(df):
    
    #use isnull() pandas function, python None does not work in this case
    
    new_df = df[(df['FIPS'].isnull() == False) ]
    
    return new_df

In [13]:
def del_col(df, col_name):
    
    df = df.drop(col_name, axis=1)
    
    return df

In [14]:
def change_data_type(df, col_list, data_type_list): #first element of col_list correlates to first element of data_type_list
    
    for i in range(0, len(col_list)):
        df[col_list[i]] = df[col_list[i]].astype(data_type_list[i])
        
    return df

In [15]:
def merge_col(df1, df2, col_list):

    merged_df = pd.DataFrame()
    
    #for i in range(0,len(df_list)-1):
    merged_df = df1.merge(df2, how='outer', on=col_list) #col_list type - string  
        
    return merged_df

In [16]:
def write_csv_file(df, file_name):
    
    import os as os 
    import os.path
    
    #check if file exists
    if os.path.exists(file_name):
        file = open(file_name, "r")
        file.close()
        #os.remove(file_name) #removing the existing file from the folder 
      
    df.to_csv(file_name) #creating a new csv file
    
    return

### Main

In [ ]:
file_sheet_list = [['2010 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2011 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2012 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2013 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2014 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2015 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2016 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2017 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2018 County Health Rankings Data.xls', 'Ranked Measure Data'],
                   ['2019 County Health Rankings Data.xls', 'Ranked Measure Data']]

replace_header = True

#year_list = ['2010', '2011', '2012', '2013']
year_list = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
year_col_num = 3

col_list = ['FIPS', 'State', 'County', 'Year', 'Physically Unhealthy Days','Mentally Unhealthy Days',
                  '% LBW','% Smokers','% Obese','% Physically Inactive','% Excessive Drinking','PCP Rate','High School Grad Rate',
                  '% Some College Education','% Unemployed','% Uninsured','% Children in Poverty','Income Ratio',
                   '% Single-Parent Households'] #string

merged_df = pd.DataFrame()

for i in range(0,len(file_sheet_list)): 
    
    df = extract_clean_df(file_sheet_list[i][0], file_sheet_list[i][1], replace_header, year_list[i], year_col_num, col_list)
    merged_df = merge_row(merged_df, df)

merged_df = del_null_row(merged_df)    

csv_file_name = 'Sociodemo.csv'
write_csv_file(merged_df,csv_file_name)

## Merge with Outside Data

### Disease & Air Pollution Merge

In [ ]:
#creating new dataframes so they don't duplicate after each run 
sociodem_df = pd.DataFrame()
dis_air_df = pd.DataFrame()

In [ ]:
sociodem_df = pd.read_csv("Sociodemo.csv")
dis_air_df = pd.read_csv("AllData_Andrew.csv")

In [ ]:
col_list_1 = ['FIPS','State','County', 'Year']
data_type_list_1 = [str, str, str, str]
df_list_1 = [dis_air_df, sociodem_df]

df_list_1[0] = change_data_type(df_list_1[0], col_list_1, data_type_list_1)
df_list_1[1] = change_data_type(df_list_1[1], col_list_1, data_type_list_1)

In [ ]:
all_data_df = pd.DataFrame()

In [ ]:
#all_data_df = dis_air_df.merge(sociodem_df, how='outer', on=['FIPS', 'State', 'County', 'Year'])

all_data_df = merge_col(df_list_1[0], df_list_1[1], col_list_1)
all_data_df = del_col(all_data_df, 'Unnamed: 0')
all_data_df = del_col(all_data_df, 'Index')

write_csv_file(all_data_df, 'dis_air_sociodem.csv')

del all_data_df #free memory

#### Clean Rows

In [ ]:
def del_year_row(df, year_list):
     
    for i in year_list:
        df = df.loc[df['Year']!= i]
    
    return df

In [ ]:
dis_air_sociodem_df = pd.DataFrame()
dis_air_sociodem_df = pd.read_csv('dis_air_sociodem.csv')

In [ ]:
year_list = [1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2020, 2021, 2022]

In [ ]:
dis_air_sociodem_df = del_year_row(dis_air_sociodem_df, year_list)
write_csv_file(dis_air_sociodem_df, 'dis_air_sociodem.csv')

In [ ]:
del dis_air_sociodem_df

#### Merging Poverty & Demographics Data

In [212]:
sociodem_df = pd.DataFrame()
sociodem_df = pd.read_csv('Sociodemo.csv')

pov_df = pd.DataFrame()
pov_df = pd.read_csv('poverty_2010-2019.csv')

pov_df['Year'] = pov_df['Year'].astype(str)
pov_df['FIPS'] = pov_df['FIPS'].astype(str)

sociodem_df['FIPS'] = sociodem_df['FIPS'].astype(str)
sociodem_df['Year'] = sociodem_df['Year'].astype(str)

pov_df = pov_df[['FIPS', 'Year', 'Poverty Percent, All Ages']].copy()

In [213]:
sociodem_pov_df = sociodem_df.merge(pov_df, how='outer', on=['FIPS', 'Year'])
sociodem_pov_df = sociodem_pov_df[sociodem_pov_df['State'].isnull()==False]
sociodem_pov_df

,FIPS,State,County,Year,Physically Unhealthy Days,Mentally Unhealthy Days,% LBW,% Smokers,% Obese,% Physically Inactive,% Excessive Drinking,PCP Rate,High School Grad Rate,% Some College Education,% Unemployed,% Uninsured,% Children in Poverty,Income Ratio,% Single-Parent Households,"Poverty Percent, All Ages"
0,1001,Alabama,Autauga,2010,5.480000,4.140000,8.830000,28.140000,30.0,,13.590000,52.224775,71.518987,20.643627,4.600000,14.000000,13.8,40.7,8.662107,11.9
1,1003,Alabama,Baldwin,2010,3.570000,4.060000,8.770000,23.090000,24.5,,17.850000,70.741370,67.043702,26.321226,4.100000,20.900000,15.3,44.5,8.600628,13.3
2,1005,Alabama,Barbour,2010,6.110000,3.840000,11.000000,22.680000,36.4,,9.840000,40.744262,56.342183,11.186903,7.800000,15.100000,34.1,47.4,14.355162,25.3
3,1007,Alabama,Bibb,2010,4.220000,5.310000,9.110000,NaN,31.7,,9.540000,42.105263,60.383387,11.000845,5.200000,17.700000,24.2,39.8,5.125778,20.9
4,1009,Alabama,Blount,2010,5.620000,4.470000,7.260000,23.420000,31.5,,6.140000,19.987281,73.013493,11.618125,4.100000,20.800000,18.5,41.1,7.120596,16.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31402,56037,Wyoming,Sweetwater,2019,3.536556,3.543546,9.420457,17.566943,30.2,25.6,20.325111,33.963550,80.067568,59.295545,4.550299,12.750852,13.4,4.393950545,23.461861,8.3
31403,56039,Wyoming,Teton,2019,3.166316,3.072253,7.473928,14.137266,13.6,12,20.451146,120.736490,93.464052,82.004243,2.987005,14.979313,6.9,3.930119004,24.042601,6
31404,56041,Wyoming,Uinta,2019,3.677538,3.699922,9.647495,18.180586,31.8,23.5,16.058559,38.511530,84.687500,55.891810,4.497702,12.568976,11.8,4.635869103,21.645737,8.5
31405,56043,Wyoming,Washakie,2019,3.601687,3.479694,7.320872,15.607573,29.7,26,16.342995,48.573160,86.290323,60.132159,4.082625,16.708229,14.0,3.611459192,26.629611,11.1


In [214]:
write_csv_file(sociodem_pov_df,'sociodempov.csv')

#### Merging with Race & Housing 

In [262]:
race_df = pd.DataFrame()
race_df = pd.read_csv('Race_Sociodemo.csv')

race_df['Year'] = race_df['Year'].astype(str)
race_df['FIPS'] = race_df['FIPS'].astype(str)

race_df = race_df.drop(columns=['State','County','Population'])

#race_df.info()
race_df[sociodem_pov_temp_df['FIPS']==10001]

C:\Users\Hannah Chang\AppData\Local\Temp\ipykernel_18096\1287209824.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  race_df[sociodem_pov_temp_df['FIPS']==10001]


,FIPS,Year,% Age < 18,% Age 65 and over,% African American,% Asian,% American Indian/Alaskan Native,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Median Household Income,% high housing costs
314,10001,2011,25.200000,13.400000,23.300000,2.000000,0.600000,0.100000,4.600000,NaN,3.200000,52.100000,35.500000,55179.0,35.200000
3455,10001,2012,25.200000,13.400000,23.300000,2.000000,0.600000,0.100000,4.600000,NaN,3.493543,52.100000,35.500000,51490.0,34.985016
6596,10001,2013,24.474926,13.887305,23.524273,2.196149,0.687358,0.091001,5.993909,64.937452,1.652393,51.821226,26.969380,51045.0,36.464280
9737,10001,2014,24.179423,14.465536,23.614950,2.167930,0.724660,0.088314,6.071897,64.708231,1.445596,51.844616,26.969380,51695.0,NaN
12878,10001,2015,23.803537,15.068234,23.942839,2.214667,0.723662,0.096803,6.621571,63.708859,1.418057,51.805615,26.969380,53811.0,NaN
16019,10001,2016,23.558176,15.562223,23.961695,2.274591,0.736102,0.113381,6.887148,63.325716,1.124774,51.772518,26.969380,54271.0,NaN
19160,12001,2017,18.025957,12.888708,19.948147,6.131618,0.335431,0.075010,9.191657,62.180148,1.068862,51.628302,21.193437,47023.0,NaN
22297,9015,2018,20.130474,15.680942,1.830591,1.452768,0.665278,0.086925,11.528332,83.096082,2.649310,50.435486,49.761036,60869.0,NaN
25439,9015,2019,19.807664,16.249710,1.846870,1.501388,0.654010,0.081644,11.684528,82.852207,2.415149,50.311536,49.761036,62606.0,NaN


In [216]:
sociodem_pov_df = sociodem_pov_df.merge(race_df, how='outer', on=['FIPS', 'Year'])
sociodem_pov_df

,FIPS,State,County,Year,Physically Unhealthy Days,Mentally Unhealthy Days,% LBW,% Smokers,% Obese,% Physically Inactive,...,% Asian,% American Indian/Alaskan Native,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Median Household Income,% high housing costs
0,1001,Alabama,Autauga,2010,5.480000,4.140000,8.830000,28.140000,30.0,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1003,Alabama,Baldwin,2010,3.570000,4.060000,8.770000,23.090000,24.5,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1005,Alabama,Barbour,2010,6.110000,3.840000,11.000000,22.680000,36.4,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1007,Alabama,Bibb,2010,4.220000,5.310000,9.110000,NaN,31.7,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1009,Alabama,Blount,2010,5.620000,4.470000,7.260000,23.420000,31.5,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31402,56037,Wyoming,Sweetwater,2019,3.536556,3.543546,9.420457,17.566943,30.2,25.6,...,0.833831,1.534433,0.142417,16.249368,79.358662,1.582845,48.454082,10.916313,75590.0,NaN
31403,56039,Wyoming,Teton,2019,3.166316,3.072253,7.473928,14.137266,13.6,12,...,1.229314,0.906942,0.133247,14.983881,81.702128,4.042033,48.098001,46.430920,90145.0,NaN
31404,56041,Wyoming,Uinta,2019,3.677538,3.699922,9.647495,18.180586,31.8,23.5,...,0.492803,1.395462,0.175653,9.163211,87.474994,0.911363,49.573067,43.095937,67404.0,NaN
31405,56043,Wyoming,Washakie,2019,3.601687,3.479694,7.320872,15.607573,29.7,26,...,0.756448,1.723710,0.111607,14.248512,82.428075,0.346198,49.466766,35.954529,57989.0,NaN


In [217]:
write_csv_file(sociodem_pov_df, 'sociodempov.csv')

#### Merging with Temperature

In [218]:
sociodem_pov_temp_df = pd.DataFrame()
sociodem_pov_temp_df = pd.read_csv('sociodempov.csv')

temp_df = pd.DataFrame()
temp_df = pd.read_csv("seasonal_temp_data_2000-2016.csv")

temp_df['FIPS'] = temp_df['FIPS'].astype(str)
temp_df['Year'] = temp_df['Year'].astype(str)

C:\Users\Hannah Chang\AppData\Local\Temp\ipykernel_18096\3434578485.py:2: DtypeWarning: Columns (10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  sociodem_pov_temp_df = pd.read_csv('sociodempov.csv')


In [219]:
sociodem_pov_temp_df = sociodem_pov_df.merge(temp_df, how='outer', on=['FIPS', 'Year'])
sociodem_pov_temp_df = sociodem_pov_temp_df[sociodem_pov_temp_df['State'].isnull() ==False]
sociodem_pov_temp_df = sociodem_pov_temp_df.drop(columns=['State', 'County'])
sociodem_pov_temp_df

,FIPS,Year,Physically Unhealthy Days,Mentally Unhealthy Days,% LBW,% Smokers,% Obese,% Physically Inactive,% Excessive Drinking,PCP Rate,...,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Median Household Income,% high housing costs,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax
0,1001,2010,5.480000,4.140000,8.830000,28.140000,30.0,,13.590000,52.224775,...,NaN,NaN,NaN,NaN,NaN,NaN,307.301620,95.930844,284.516593,85.556321
1,1003,2010,3.570000,4.060000,8.770000,23.090000,24.5,,17.850000,70.741370,...,NaN,NaN,NaN,NaN,NaN,NaN,306.362271,95.062925,287.316057,88.375102
2,1005,2010,6.110000,3.840000,11.000000,22.680000,36.4,,9.840000,40.744262,...,NaN,NaN,NaN,NaN,NaN,NaN,307.031201,96.743853,285.548278,86.601035
3,1007,2010,4.220000,5.310000,9.110000,NaN,31.7,,9.540000,42.105263,...,NaN,NaN,NaN,NaN,NaN,NaN,307.045058,96.478311,283.897197,85.695587
4,1009,2010,5.620000,4.470000,7.260000,23.420000,31.5,,6.140000,19.987281,...,NaN,NaN,NaN,NaN,NaN,NaN,306.453123,96.377585,281.889789,84.789725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31402,56037,2019,3.536556,3.543546,9.420457,17.566943,30.2,25.6,20.325111,33.963550,...,79.358662,1.582845,48.454082,10.916313,75590.0,NaN,NaN,NaN,NaN,NaN
31403,56039,2019,3.166316,3.072253,7.473928,14.137266,13.6,12,20.451146,120.736490,...,81.702128,4.042033,48.098001,46.430920,90145.0,NaN,NaN,NaN,NaN,NaN
31404,56041,2019,3.677538,3.699922,9.647495,18.180586,31.8,23.5,16.058559,38.511530,...,87.474994,0.911363,49.573067,43.095937,67404.0,NaN,NaN,NaN,NaN,NaN
31405,56043,2019,3.601687,3.479694,7.320872,15.607573,29.7,26,16.342995,48.573160,...,82.428075,0.346198,49.466766,35.954529,57989.0,NaN,NaN,NaN,NaN,NaN


In [261]:
sociodem_pov_temp_df[sociodem_pov_temp_df['FIPS']==10001]

,FIPS,Year,Physically Unhealthy Days,Mentally Unhealthy Days,% LBW,% Smokers,% Obese,% Physically Inactive,% Excessive Drinking,PCP Rate,...,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Median Household Income,% high housing costs,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax
314,10001,2010,3.680000,3.610000,9.240000,23.920000,31.2,,15.130000,50.944213,...,NaN,NaN,NaN,NaN,NaN,NaN,303.868674,88.387689,278.272359,84.192369
3455,10001,2011,3.700000,3.600000,9.340000,23.000000,32.4,,16.300000,49.520233,...,NaN,3.200000,52.100000,35.50000,55179.0,35.200000,304.583917,91.653776,278.553625,81.011010
6596,10001,2012,3.800000,3.600000,9.200000,22.300000,32.7,28.2,16.000000,49.520233,...,NaN,3.493543,52.100000,35.50000,51490.0,34.985016,305.124542,89.518368,283.837644,83.452235
9737,10001,2013,3.800000,3.600000,8.800000,21.900000,32.7,28.2,16.400000,51.573930,...,64.937452,1.652393,51.821226,26.96938,51045.0,36.464280,303.193939,94.577484,282.429366,83.507429
12878,10001,2014,3.700000,3.500000,8.600000,21.200000,33.8,27.6,16.100000,49.747020,...,64.708231,1.445596,51.844616,26.96938,51695.0,NaN,301.739543,87.214805,279.748024,85.535824
16019,10001,2015,3.700000,3.500000,8.528471,21.200000,32.7,27.4,16.100000,60.849750,...,63.708859,1.418057,51.805615,26.96938,53811.0,NaN,302.029036,93.031164,278.386625,85.130999
19160,10001,2016,3.600000,3.800000,8.476975,18.200000,33.2,28.1,15.300000,60.797090,...,63.325716,1.124774,51.772518,26.96938,54271.0,NaN,303.174632,93.299935,283.159273,84.397250
22297,10001,2017,3.475073,4.035169,8.451250,19.019157,32.8,27.8,15.421028,47.096580,...,62.845107,1.219181,51.795912,26.96938,55678.0,NaN,NaN,NaN,NaN,NaN
25439,10001,2018,3.535874,3.681000,8.882616,18.170641,33.6,30.3,16.445973,45.524480,...,62.411412,1.151782,51.843823,26.96938,53994.0,NaN,NaN,NaN,NaN,NaN
28581,10001,2019,3.535874,3.681000,9.030211,18.170641,33.8,29.5,16.445973,44.615530,...,61.714473,1.268547,51.821585,26.96938,57112.0,NaN,NaN,NaN,NaN,NaN


In [220]:
write_csv_file(sociodem_pov_temp_df,"sociodempovtemp.csv")

#### Merging with Disease/Pollution

In [254]:
dis_air_df = pd.DataFrame()
sociodem_pov_temp_df = pd.DataFrame()

dis_air_df = pd.read_csv("AllData_Andrew.csv")
sociodem_pov_temp_df = pd.read_csv("sociodempovtemp.csv")

dis_air_df = dis_air_df.drop(columns='Index')

C:\Users\Hannah Chang\AppData\Local\Temp\ipykernel_18096\1462951649.py:5: DtypeWarning: Columns (7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  sociodem_pov_temp_df = pd.read_csv("sociodempovtemp.csv")


In [255]:
dis_air_df = dis_air_df[(dis_air_df['Year']== 2010) | (dis_air_df['Year']== 2011) |
                        (dis_air_df['Year']== 2012) | (dis_air_df['Year']== 2013) |
                        (dis_air_df['Year']== 2014) | (dis_air_df['Year']== 2015) |
                        (dis_air_df['Year']== 2016) | (dis_air_df['Year']== 2017) |
                        (dis_air_df['Year']== 2018) | (dis_air_df['Year']== 2019)] 

In [260]:
sociodem_pov_temp_df[sociodem_pov_temp_df['FIPS']==10001]

,FIPS,Year,Physically Unhealthy Days,Mentally Unhealthy Days,% LBW,% Smokers,% Obese,% Physically Inactive,% Excessive Drinking,PCP Rate,...,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Median Household Income,% high housing costs,summer_tmmx,summer_rmax,winter_tmmx,winter_rmax
314,10001,2010,3.680000,3.610000,9.240000,23.920000,31.2,,15.130000,50.944213,...,NaN,NaN,NaN,NaN,NaN,NaN,303.868674,88.387689,278.272359,84.192369
3455,10001,2011,3.700000,3.600000,9.340000,23.000000,32.4,,16.300000,49.520233,...,NaN,3.200000,52.100000,35.50000,55179.0,35.200000,304.583917,91.653776,278.553625,81.011010
6596,10001,2012,3.800000,3.600000,9.200000,22.300000,32.7,28.2,16.000000,49.520233,...,NaN,3.493543,52.100000,35.50000,51490.0,34.985016,305.124542,89.518368,283.837644,83.452235
9737,10001,2013,3.800000,3.600000,8.800000,21.900000,32.7,28.2,16.400000,51.573930,...,64.937452,1.652393,51.821226,26.96938,51045.0,36.464280,303.193939,94.577484,282.429366,83.507429
12878,10001,2014,3.700000,3.500000,8.600000,21.200000,33.8,27.6,16.100000,49.747020,...,64.708231,1.445596,51.844616,26.96938,51695.0,NaN,301.739543,87.214805,279.748024,85.535824
16019,10001,2015,3.700000,3.500000,8.528471,21.200000,32.7,27.4,16.100000,60.849750,...,63.708859,1.418057,51.805615,26.96938,53811.0,NaN,302.029036,93.031164,278.386625,85.130999
19160,10001,2016,3.600000,3.800000,8.476975,18.200000,33.2,28.1,15.300000,60.797090,...,63.325716,1.124774,51.772518,26.96938,54271.0,NaN,303.174632,93.299935,283.159273,84.397250
22297,10001,2017,3.475073,4.035169,8.451250,19.019157,32.8,27.8,15.421028,47.096580,...,62.845107,1.219181,51.795912,26.96938,55678.0,NaN,NaN,NaN,NaN,NaN
25439,10001,2018,3.535874,3.681000,8.882616,18.170641,33.6,30.3,16.445973,45.524480,...,62.411412,1.151782,51.843823,26.96938,53994.0,NaN,NaN,NaN,NaN,NaN
28581,10001,2019,3.535874,3.681000,9.030211,18.170641,33.8,29.5,16.445973,44.615530,...,61.714473,1.268547,51.821585,26.96938,57112.0,NaN,NaN,NaN,NaN,NaN


In [256]:
dis_air_df

,FIPS,Year,State,County,Population,Latitude,Longitude,Alzheimers Deaths,Cerebrovascular Disease Deaths,Chronic liver disease and cirrhosis Deaths,...,CO 8-hour 1971,Lead 3-Month 2009,NO2 1-hour 2010,NO2 Annual 1971,Ozone 8-hour 2015,PM10 24-hour 2006,PM25 24-hour 2012,PM25 Annual 2012,SO2 1-hour 2010,SO2 3-hour 1971
11,10001,2010,Delaware,Kent,162310.0,39.086,-75.5686,34.0,57.0,20.0,...,NaN,NaN,NaN,NaN,0.049462,NaN,9.539514,9.539514,NaN,NaN
12,10001,2011,Delaware,Kent,164834.0,39.086,-75.5686,42.0,51.0,17.0,...,NaN,NaN,NaN,NaN,0.046864,NaN,8.820699,8.820699,NaN,NaN
13,10001,2012,Delaware,Kent,167626.0,39.086,-75.5686,40.0,80.0,NaN,...,NaN,NaN,NaN,NaN,0.049642,NaN,8.205308,8.205308,NaN,NaN
14,10001,2013,Delaware,Kent,169416.0,39.086,-75.5686,29.0,61.0,11.0,...,NaN,NaN,NaN,NaN,0.044270,NaN,8.018102,8.018102,NaN,NaN
15,10001,2014,Delaware,Kent,171987.0,39.086,-75.5686,21.0,76.0,22.0,...,NaN,NaN,NaN,NaN,0.043709,NaN,8.246808,8.246808,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75400,9015,2015,Connecticut,Windham,116573.0,41.830,-71.9874,37.0,56.0,17.0,...,NaN,NaN,NaN,NaN,0.043469,NaN,NaN,NaN,NaN,NaN
75401,9015,2016,Connecticut,Windham,116192.0,41.830,-71.9874,36.0,31.0,19.0,...,NaN,NaN,NaN,NaN,0.043234,NaN,NaN,NaN,NaN,NaN
75402,9015,2017,Connecticut,Windham,116359.0,41.830,-71.9874,37.0,41.0,10.0,...,NaN,NaN,NaN,NaN,0.040923,NaN,NaN,NaN,NaN,NaN
75403,9015,2018,Connecticut,Windham,117027.0,41.830,-71.9874,26.0,34.0,14.0,...,NaN,NaN,NaN,NaN,0.041871,NaN,NaN,NaN,NaN,NaN


In [257]:
disair_sociodem_pov_temp_df = dis_air_df.merge(sociodem_pov_temp_df, how='outer', on=['FIPS', 'Year'])
disair_sociodem_pov_temp_df = disair_sociodem_pov_temp_df[disair_sociodem_pov_temp_df['State'].isnull()==False]
disair_sociodem_pov_temp_df['% Age < 18'].head(20)

0           NaN
1     25.200000
2     25.200000
3     24.474926
4     24.179423
5     23.803537
6     23.558176
7     23.253790
8     23.039919
9     22.967470
10          NaN
11    23.800000
12    23.800000
13    22.842182
14    22.653806
15    22.334650
16    22.174725
17    21.951977
18    21.864963
19    21.685516
Name: % Age < 18, dtype: float64

In [248]:
write_csv_file(disair_sociodem_pov_temp_df, 'AllData_w_demo.csv')

### New Disease Dataset Merge

In [292]:
demo = pd.DataFrame() 
newdis = pd.DataFrame() 

demo = pd.read_csv('AllData_w_demo.csv')
newdis = pd.read_csv('AllDataNew.csv')

C:\Users\Hannah Chang\AppData\Local\Temp\ipykernel_18096\1866782900.py:4: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  demo = pd.read_csv('AllData_w_demo.csv')


In [285]:
newdis_df = newdis.drop(newdis.loc[:,'age_pct_0_14':'winter_tmmx'], axis=1)
newdis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69124 entries, 0 to 69123
Data columns (total 45 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   Index                                                                           69124 non-null  object 
 1   FIPS                                                                            69124 non-null  int64  
 2   Year                                                                            69124 non-null  int64  
 3   State                                                                           69124 non-null  object 
 4   County                                                                          69124 non-null  object 
 5   Population                                                                      68996 non-null  float64
 6   Latitude      

In [293]:
demo_df = demo.drop(demo.iloc[:,2:30], axis=1)
demo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31420 entries, 0 to 31419
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   FIPS                                      31420 non-null  int64  
 1   Year                                      31420 non-null  int64  
 2   Physically Unhealthy Days                 29836 non-null  float64
 3   Mentally Unhealthy Days                   29370 non-null  float64
 4   % LBW                                     29746 non-null  float64
 5   % Smokers                                 27925 non-null  float64
 6   % Obese                                   31364 non-null  float64
 7   % Physically Inactive                     31364 non-null  object 
 8   % Excessive Drinking                      27392 non-null  float64
 9   PCP Rate                                  30261 non-null  float64
 10  High School Grad Rate             

In [296]:
AllData = pd.merge(newdis_df, demo_df, how='outer', on=['FIPS', 'Year'])
AllData = AllData.drop(columns=['Index'])

#AllData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69124 entries, 0 to 69123
Data columns (total 77 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   FIPS                                                                            69124 non-null  int64  
 1   Year                                                                            69124 non-null  int64  
 2   State                                                                           69124 non-null  object 
 3   County                                                                          69124 non-null  object 
 4   Population                                                                      68996 non-null  float64
 5   Latitude                                                                        69102 non-null  float64
 6   Longitude     

In [297]:
write_csv_file(AllData, 'AllData_7-12.csv')

## Debug Space

In [ ]:
merged_df.tail()